In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

try:
    if torch.cuda.is_available():
        import torch.cuda.amp
        MIXED_PRECISION = True
    else:
        MIXED_PRECISION = False  # not installed
except:
    print('Pytorch>=1.6 is recommended for faster mixed precision training')
    MIXED_PRECISION = False  # not installed
class build_model():
    def __init__(self, network):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        #self.network = PLSnet(2, 1)
        self.network = network
        #self.init_weights(self.network)
        self.optimizer = torch.optim.Adam(self.network.parameters(), lr=1, betas=[0.9, 0.999])
        #self.optimizer = pytorch_lamb.Lamb(self.network.parameters(), lr=0.002, betas=(.9, .999), adam=True)
        #self.optimizer = pytorch_lamb.LARS(self.network.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4, eta=0.001, max_epoch=200)
        self.network.to(self.device)
        if MIXED_PRECISION:
            self.scaler = torch.cuda.amp.GradScaler()
    def __call__(self,*inputs):
        if self.network.training:
            return self.network(*inputs)
        else:
            with torch.no_grad():
                return self.network(*inputs)
    def train(self):
        self.network.train()
    def eval(self):
        self.network.eval()
    def set_lr(self, new_lr):
        for g in self.optimizer.param_groups:
            g['lr'] = new_lr
    def init_weights(self, m):
        for child in m.children():
            if isinstance(child, torch.nn.Module):
                if hasattr(child, 'weight'):
                    torch.nn.init.normal_(child.weight, 0.0, 0.01)
                    #torch.nn.init.xavier_normal_(child.weight) # sqrt(1/前の層のノード数)　を標準偏差とする。Xivier初期化
                    #torch.nn.init.kaiming_normal_(child.weight) # sqrt(2/前の層のノード数)　を標準偏差とする。He初期化
                if hasattr(child, 'bias'):
                    if child.bias is not None: torch.nn.init.normal_(child.bias, 0.0, 0.01)
                self.init_weights(child)
    def update(self, loss, do_step=True): # 勾配を貯めたい場合は do_step = False　にする。
        if self.network.training:
            self.optimizer.zero_grad()
            if MIXED_PRECISION:
                self.scaler.scale(loss).backward()
                self.scaler.step(self.optimizer)
                if do_step: self.scaler.update()
            else:
                loss.backward()
                if do_step: self.optimizer.step()
    def save(self, fn=None):
        _fn = 'network_params.pth' if fn is None else fn
        torch.save(self.network.state_dict(), _fn)
    def load(self, fn=None):
        _fn = 'network_params.pth' if fn is None else fn
        self.network.load_state_dict(torch.load(_fn))
    def gradient_penalty(self, loss): # 勾配の二乗をロスに加えるサンプルコード。 torch.cuda.ampが使えるときに使用可能
        grad_params = torch.autograd.grad(loss, self.network.parameters(), create_graph=True)
        grad_norm = 0
        for grad in grad_params:
            grad_norm += grad.pow(2).sum()
        grad_norm = grad_norm.sqrt()
        loss = loss + grad_norm

class curve(nn.Module):
    def __init__(self):
        super(curve, self).__init__()
        self.fvc = nn.Parameter(torch.Tensor(np.zeros(150)))
    def set_init(self, fvc_true, weeks):
        xp = weeks.cpu().numpy()+12
        fp = fvc_true.cpu().numpy()
        fp_start = fp[0] - (fp[0]-fp[-1])/(xp[0]-xp[-1])*(xp[0] - 0)**0.5
        fp_end = fp[-1] + (fp[0]-fp[-1])/(xp[0]-xp[-1])*(150 - xp[-1])**0.5
        fp = np.concatenate([[fp_start], fp, [fp_end]])
        xp = np.concatenate([[0], xp, [149]])
        init = np.interp(np.arange(150), xp, fp)
        self.fvc.data = torch.Tensor(init)
    def calc_loss(self, fvc_true, weeks):
        inner_idx = weeks.long()+12
        fvc_pred = self.fvc[inner_idx]
        l1_loss = (fvc_true - fvc_pred).abs()
        d1 = self.fvc[:-1] - self.fvc[1:]
        d2 = d1[:-1] - d1[1:]
        d2_5 = d1[:-5] - d1[5:]
        d2_10 = d1[:-10] - d1[10:]
        d2_20 = d1[:-20] - d1[20:]
        d1 = torch.cat([d1, d1[-1:]])
        d2 = torch.cat([d2[:1], d2, d2[-1:]])
        d2_5 = torch.cat([d2_5[:3], d2_5, d2_5[-3:]])
        d2_10 = torch.cat([d2_10[:5], d2_10, d2_10[-6:]])
        d2_20 = torch.cat([d2_20[:10], d2_20, d2_20[-11:]])
        d0_loss = self.fvc.square()
        d1_loss = d1.square()
        d2_loss = d2.square()
        d2_5_loss = d2_5.square()
        d2_10_loss = d2_10.square()
        d2_20_loss = d2_20.square()
        loss_reg = torch.stack([d1_loss, d2_loss, d2_5_loss, d2_10_loss, d2_20_loss], 1)
        inner_loss = l1_loss.mean() + torch.Tensor([1,1,1,1,0]) @ loss_reg[inner_idx.min():inner_idx.max()-9].mean(0)
        outer1_loss = torch.Tensor([1,1,1,1,0]) @ loss_reg[:inner_idx.min()+10].mean(0)
        outer2_loss = torch.Tensor([0,10,5,1,1]) @ loss_reg[inner_idx.max()-9:].mean(0)
        return outer1_loss + inner_loss + outer2_loss

In [ ]:
""" data load"""
root_dir = Path('../input/osic-pulmonary-fibrosis-progression')

ds = pd.read_csv(Path(root_dir)/"train.csv")
ds.drop_duplicates(keep=False, inplace=True, subset=['Patient', 'Weeks'])
ds.set_index('Patient')
ps = ds.Patient.unique()

In [ ]:
""" Sample curve fitting """
FVC_pred = np.zeros((len(ps),150))
for i in range(len(ps)):
    #i = 23
    patient = ps[i]
    p = ds[ds.Patient==patient].sort_values(by='Weeks')
    model = build_model(curve())
    self = model.network
    fvc_true = torch.Tensor(p.FVC.values).to(model.device)
    weeks = torch.Tensor(p.Weeks.values).to(model.device)
    model.network.set_init(fvc_true, weeks)
    model.set_lr(1)
    for _ in range(300):
        loss = model.network.calc_loss(fvc_true, weeks)
        #print(loss)
        model.update(loss)
    smooth_fvc = model.network.fvc.detach().cpu().numpy()
    FVC_pred[i] = smooth_fvc

    print('\r{}'.format(i),end='')

In [ ]:
def seek_sigma(FVC_pred, FVC_true):
    delta = np.abs(FVC_pred - FVC_true)[None]
    sigma = (delta.mean() + np.arange(300)/10 - 3)[:,None]
    score = -np.sqrt(2) * delta / sigma - np.log(np.sqrt(2) * sigma)
    score = score.mean(1)
    return sigma[:,0][score==score.min()][0]
def seek_sigma_by_delta(delta):
    delta = np.array(delta)[None]
    sigma = (delta.mean() + np.arange(300)/10 - 3)[:,None]
    score = -np.sqrt(2) * delta / sigma - np.log(np.sqrt(2) * sigma)
    score = score.mean(1)
    return sigma[:,0][score==score.min()][0], score.min()
sigma = np.zeros(len(ps))
score = np.zeros(len(ps))
for i in range(len(ps)):
    #sigma[i],score[i] = seek_sigma_by_delta(deltas[i])
    patient = ps[i]
    p = ds[ds.Patient==patient].sort_values(by='Weeks')
    fvc_true = p.FVC.values
    fvc_pred = FVC_pred[i][p.Weeks.values]
    sigma[i] = seek_sigma(fvc_pred, fvc_true)
    print('\r{}'.format(i+1),end='')

In [ ]:
submission = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/sample_submission.csv')
submission['Patient'] = submission['Patient_Week'].apply(lambda x: x.split('_')[0])
submission['predict_Week'] = submission['Patient_Week'].apply(lambda x: x.split('_')[1]).astype(int)

test = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/test.csv')\
        .rename(columns={'Weeks': 'base_Week', 'FVC': 'base_FVC', 'Percent': 'base_Percent', 'Age': 'base_Age'})
test = submission.drop(columns=['FVC', 'Confidence']).merge(test, on='Patient')
test['Week_passed'] = test['predict_Week'] - test['base_Week']

submission = submission.drop(columns=['Patient', 'predict_Week'])

In [ ]:
FVC_sub = []
sigma_sub = []
for r, rowitem in test.iterrows():
    patient = rowitem.Patient
    w = rowitem.predict_Week
    FVC_sub.append( FVC_pred[ps==patient][0,w+12] )
    sigma_sub.append( sigma[ps==patient][0] )
test['FVC_pred'] = FVC_sub
test['Confidence'] = 400

In [ ]:

sub = submission.drop(columns=['FVC', 'Confidence']).merge(test[['Patient_Week', 'FVC_pred', 'Confidence']], 
                                                           on='Patient_Week')
sub.columns = submission.columns
sub.to_csv('submission.csv', index=False)
sub.head()